In [2]:
import os
import glob
import numpy as np
import cv2
import random
from shapely.geometry import box, Polygon
from shapely.affinity import rotate
from PIL import Image

def get_rotated_box(box, angle, center, img_shape):
    """Get rotated box for a given angle and clip to image boundaries"""
    rect = Polygon(box)
    rotated_rect = rotate(rect, angle, origin=center)
    clipped_rotated_rect = []
    for x, y in rotated_rect.exterior.coords[:]:
        x = np.clip(x, 0, img_shape[1] - 1)
        y = np.clip(y, 0, img_shape[0] - 1)
        clipped_rotated_rect.append((x, y))
    return clipped_rotated_rect

def rotate_image_and_labels(image_path, labels_path, save_dir_images, save_dir_labels):
    """Rotate image and labels"""
    for angle in range(3):
        angle = random.uniform(0, 360)
        image = Image.open(image_path)
        image_rotated = image.rotate(angle)
        image_rotated.save(os.path.join(save_dir_images, f"{os.path.basename(image_path).split('.')[0]}_rotated{angle}.png"))
        
        with open(labels_path, 'r') as f:
            lines = f.readlines()
        with open(os.path.join(save_dir_labels, f"{os.path.basename(labels_path).split('.')[0]}_rotated{angle}.txt"), 'w') as f:
            for line in lines:
                coords = list(map(float, line.split(',')[:8]))
                coords = [(coords[i], coords[i + 1]) for i in range(0, len(coords), 2)]
                new_coords = get_rotated_box(coords, angle, (image.width / 2, image.height / 2))
                new_coords = [val for sublist in new_coords for val in sublist]
                f.write(','.join(map(str, new_coords)) + line[line.find('container'):])

def flip_image_and_labels(image_path, labels_path, save_dir_images, save_dir_labels):
    """Flip image and labels"""
    image = Image.open(image_path)
    image_flipped = image.transpose(Image.FLIP_TOP_BOTTOM)
    image_flipped.save(os.path.join(save_dir_images, f"{os.path.basename(image_path).split('.')[0]}_flipped.png"))

    with open(labels_path, 'r') as f:
        lines = f.readlines()
    with open(os.path.join(save_dir_labels, f"{os.path.basename(labels_path).split('.')[0]}_flipped.txt"), 'w') as f:
        for line in lines:
            coords = list(map(float, line.split(',')[:8]))
            for i in range(1, 8, 2):
                coords[i] = image.height - coords[i]
            f.write(','.join(map(str, coords)) + line[line.find('container'):])

# Get all image and label paths
image_paths = glob.glob('train_images/*.png')
label_paths = [path.replace('train_images', 'train_labels_dota').replace('.png', '.txt') for path in image_paths]

# Create directories to save flipped and rotated images and labels
os.makedirs('augmented_images', exist_ok=True)
os.makedirs('augmented_labels', exist_ok=True)

# Perform flipping and rotating
for image_path, labels_path in zip(image_paths, label_paths):
    flip_image_and_labels(image_path, labels_path, 'augmented_images', 'augmented_labels')
    rotate_image_and_labels(image_path, labels_path, 'augmented_images', 'augmented_labels')


ModuleNotFoundError: No module named 'cv2'